# pytorch で深層学習 2

<div style="text-align: right;">
2022/08/01 中山将伸作成<BR>
2024/10/25 講習会用に修正（AE追加）<BR>
</div>
    
1) Li-Fデータを使って回帰予測<BR>
2) Li-F電気陰性度の分布に対するオートエンコーダー適用例<BR>

In [ ]:
import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm  


# 1. Li-Fデータ(sample数1500件超)を用いて回帰分析

![image_DNN](./fig/image_1.png)

In [ ]:
path=str("./20161017_LiF_RDF_Voronoi_ME_CE_wMatminer.csv")
df_original=pd.read_csv(path, index_col=0)

df = df_original.dropna()

drop_columns = ['ID','pretty_formula','Cohesive_Energy','Decomposition_Energy','Decomp_UorS','MEvalues_type1','ME_F/S','ME_F/M/S']
df_descriptor = df.drop(drop_columns, axis=1)
target_columns = ['MEvalues_type1']
df_target = df[target_columns]

display(df_descriptor)
display(df_target)


■　保留法　test_size=0.3 として、 X, t -->  train_X, test_X, train_t, test_t  を作成

In [ ]:
X=df_descriptor.values
t=df_target.values

train_X, test_X, train_t, test_t = train_test_split(X,t,test_size=0.3)
print (len(train_X), len(test_X))

■　numpy -->  Tensor形式変換  (train_X --> train_X のまま)

In [ ]:
train_X=torch.Tensor(train_X)
train_t=torch.Tensor(train_t)
test_X=torch.Tensor(test_X)
test_t=torch.Tensor(test_t)

■　modelの定義  1784の記述子を徐々に 1つのノードに集約させる。

Tips: <BR>
  BatchNorm1d  (データの規格化)<BR>
  ReLU  (深層学習ではsigmoidよりもReLUをよく使う)<BR>
    
![ReLU](./fig/image2.png)

In [ ]:
model= nn.Sequential(
    nn.Linear(1784,128),
    nn.Sigmoid(),
    nn.Linear(128,16),
    nn.Sigmoid(),
    nn.Linear(16,1),
)

# model= nn.Sequential(
#     nn.Linear(1784,128),
#     nn.BatchNorm1d(128),
#     nn.Sigmoid(),
#     nn.Linear(128,16),
#     nn.BatchNorm1d(16),
#     nn.Sigmoid(),
#     nn.Linear(16,1),
# )

# model= nn.Sequential(
#     nn.Linear(1784,128),
#     nn.BatchNorm1d(128),
#     nn.ReLU(),
#     nn.Linear(128,64),
#     nn.BatchNorm1d(64),
#     nn.ReLU(),
#     nn.Linear(64,16),
#     nn.BatchNorm1d(16),
#     nn.ReLU(),
#     nn.Linear(16,1),
# )

■　Loss関数の定義。 MSE (mean square error)

In [ ]:
loss_fn = nn.MSELoss()   #Loss関数の定義　この場合は MSE


In [ ]:
#optimizer = optim.SGD(model.parameters(), lr=0.001)  #model.parameters() 学習させたい変数　　ln は学習率
optimizer = optim.Adam(model.parameters(), lr=0.001)  #model.parameters() 学習させたい変数　　ln は学習率

■　DNNの実行

In [ ]:
model.train()     # modelをtraining モードにする
loss_history=[]      #  lossのステップごとの推移
losstest_history=[]
for epoch in tqdm(range(1000)):
    optimizer.zero_grad()   #optimizerの初期化
    train_y=model(train_X)
    test_y=model(test_X)
    loss=loss_fn(train_y,train_t)   #Loss関数の計算
    losstest=loss_fn(test_y,test_t)
    loss_history.append(float(loss))
    losstest_history.append(float(losstest))
    loss.backward()   #傾きを計算
    optimizer.step()   #更新処理実施

plt.plot(loss_history)  #blue
plt.plot(losstest_history) #orange

In [ ]:
print (loss_fn(train_y,train_t))
plt.plot(train_y.detach().numpy(),train_t.detach().numpy(),'x')
test_y=model(test_X)
plt.plot(test_y.detach().numpy(),test_t.detach().numpy(),'+')

# 2.オートエンコーダーをつくる


![Autoencoder](./fig/image_AE.png)<BR>
注）　optimizerを二つに分けているところは自信がありません。（通常は classを使って、モデルを作ります）<BR>

In [ ]:
# Xをつくる（電気陰性度 EN_1～EN_50のみを対象にする)

df_select=df_descriptor.iloc[:,0:50]
display(df_select)

X=df_select.values #pandas > numpyにする
train_X, test_X = train_test_split(X,test_size=0.3)  #保留法　
print (len(train_X), len(test_X))
train_X=torch.Tensor(train_X)
test_X=torch.Tensor(test_X)


In [ ]:
model_encode= nn.Sequential(
    nn.Linear(50,32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Linear(32,16),
    nn.BatchNorm1d(16),
    nn.ReLU(),
    nn.Linear(16,2),
)

model_decode= nn.Sequential(
    nn.Linear(2,16),
    nn.BatchNorm1d(16),
    nn.ReLU(),
    nn.Linear(16,32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Linear(32,50)
)

optimizer1 = optim.Adam(model_encode.parameters(), lr=0.001)  #model.parameters() 学習させたい変数　　ln は学習率
optimizer2 = optim.Adam(model_decode.parameters(), lr=0.001)  #model.parameters() 学習させたい変数　　ln は学習率

In [ ]:
loss_history=[]
losstest_history=[]

for epoch in tqdm(range(3000)):
    model_encode.train()
    model_decode.train()
    optimizer1.zero_grad()   #optimizerの初期化
    optimizer2.zero_grad()   #optimizerの初期化

    y=model_encode(train_X)
    rX=model_decode(y)
    ty=model_encode(test_X)
    rtX=model_decode(ty)

    loss=loss_fn(train_X,rX)
    loss.backward()   #傾きを計算
    optimizer1.step()   #更新処理実施
    optimizer2.step()   #更新処理実施
    
    loss_history.append(float(loss))
    losstest=loss_fn(rtX,test_X)
    losstest_history.append(float(losstest))

plt.plot(loss_history)  #blue
plt.plot(losstest_history) #orange


In [ ]:
# エンコーダで特徴抽出
X = train_X
tX = test_X

y = model_encode(X)
ty = model_encode(tX)

# デコーダで再構成
rX = model_decode(y)
rtX = model_decode(ty)

# 損失を表示
print("LOSS(train):", loss_fn(rX, X).item())
print("LOSS(test):", loss_fn(rtX, tX).item())


# データの形状を確認
print("Original X shape:", X.shape)
print("Reconstructed rX shape:", rX.shape)

# 2次元または1次元データに変換してプロット
# ここでは最初のデータポイントのみをプロットする例を示しています
plt.figure()
plt.plot(X.detach().reshape(-1).numpy(),rX.detach().reshape(-1).numpy(),  '+')  # 最初のデータポイントをプロット
plt.plot(tX.detach().reshape(-1).numpy(),rtX.detach().reshape(-1).numpy(),  '+')  # 最初のデータポイントをプロット
plt.ylabel("Reconstructed X")
plt.xlabel("Original X")
plt.title("Reconstructed vs Original Data")
plt.show()


In [ ]:
print(y.shape)
plt.plot(y.detach().numpy()[:,0],y.detach().numpy()[:,1],'x')
